In [0]:
!pip install python-Levenshtein
!pip install Metaphone

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144674 sha256=4f7365898d7b0e4d261ad65b9e6bdec414181c9464a7bc895fdb8329ed6706e6
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
  Created wheel for Metaphone: filename=Metaphone-0.6-cp36-none-any.whl size=13906 sha256=49761b944334a95b9a59cbbee1167eddb3a68b29d3203d2668596dea712f1fac
  Stored in directory: /root/.cache/pip/wheels/4e/7c/f7/162d726fc83491ef23c7a0b989005024eb83a1408c96f32eaf
Successfully built Metaphone


In [0]:
import pandas as pd
from Levenshtein import distance
from metaphone import doublemetaphone
pd.set_option('display.max_columns', 500)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://mirrors.viethosting.com/apache/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
cd_df_pys = spark.read.csv("gdrive/My Drive/HNYC Collab/cd_df_final.csv", header=True)
census_df_pys = spark.read.csv("gdrive/My Drive/HNYC Collab/census_df_final.csv", header=True)

In [0]:
def levenshteinMetaphoneFunc(first_cd, first_cn, last_cd, last_cn):
    first_cd, last_cd, first_cn, last_cn = first_cd.strip(), last_cd.strip(), first_cn.strip(), last_cn.strip() 
    s1 = (first_cd + last_cd).lower()
    s2 = (first_cn + last_cn).lower()
    mf_first_cd = set(doublemetaphone(first_cd))
    mf_last_cd = set(doublemetaphone(last_cd))
    mf_first_cn = set(doublemetaphone(first_cn))
    mf_last_cn = set(doublemetaphone(last_cn))
    if len(mf_first_cd.intersection(mf_first_cn).difference({''})) > 0:
        first_good = True
    else:
        first_good = False
    if len(mf_last_cd.intersection(mf_last_cn).difference({''})) > 0:
        last_good = True
    else:
        last_good = False
    if (distance(s1, s2) < 4) and first_good and last_good:
        return True
    return False

In [0]:
from pyspark.sql.types import BooleanType
func = spark.udf.register("nameMatchCheckWithPython", levenshteinMetaphoneFunc, BooleanType())

In [0]:
merged = cd_df_pys.join(census_df_pys, 'WARD_ED', 'inner')
results = merged.filter(func(merged.CD_FIRST_NAME, merged.CENSUS_NAMEFRSTCLEAN, merged.CD_LAST_NAME, \
                          merged.CENSUS_NAMELASTB))

In [0]:
results.write.csv('gdrive/My Drive/HNYC Collab/cd_census_merged.csv')